# Test de rapidité du modèle utilisé avec TensorRT (Resnet18)

Dans ce notebook, nous allons tester la rapidité du modèle TensorRT

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras

# Création du contexte

In [ ]:
import tensorrt as trt

# Construction de la class du logger
class MyLogger(trt.ILogger):
    def __init__(self):
        trt.ILogger.__init__(self)

    def log(self, severity, msg):
        print("%s : %s" %(severity,msg))
        pass

In [ ]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

PRECISION = trt.float32

logger = MyLogger()
runtime = trt.Runtime(logger)

with open("meilleur_model_colab/meilleur_modele_colab_FP16.engine", "rb") as f:
    engine = runtime.deserialize_cuda_engine(f.read())

In [ ]:
context = engine.create_execution_context()

# Utilisation du modèle

Commençons par allouer les espaces mémoire nécessaires :

In [ ]:
# Calcul de la taille mémoire requise pour stocker les données d'entrée / sortie dans le GPU
size_input = trt.volume(engine.get_binding_shape(0))* engine.max_batch_size
size_output = trt.volume(engine.get_binding_shape(1))* engine.max_batch_size

# Allocation de mémoire de type "page-locked" sur l'hôte
input_host_mem = cuda.pagelocked_empty(size_input, trt.nptype(PRECISION))
output_host_mem = cuda.pagelocked_empty(size_output, trt.nptype(PRECISION))

# Allocation de mémoire dans la mémoire GPU
input_device_mem = cuda.mem_alloc(input_host_mem.nbytes)
output_device_mem = cuda.mem_alloc(output_host_mem.nbytes)

# Récupère les adresses en GPU des buffers entrées / sorties
bindings = [int(input_device_mem), int(output_device_mem)]

Nous allons utiliser notre modèle pour mesurer son temps de calcul et son débit pour traiter les images. Du fait de l'initialisation du GPU, nous allons mesurer ces caractéristiques après avoir utilisé le modèle un petit nombre de fois :

In [ ]:
import time

nbr_run = 100
delais = []
predictions = []

image = np.zeros((1,224, 224, 3), dtype=np.float32)

# Initialisation des calculs
print("Initialisation des calculs...")
for i in range(5):
    np.copyto(input_host_mem,image.ravel())
    cuda.memcpy_htod(input_device_mem, input_host_mem)
    context.execute(batch_size=1,bindings=bindings)
    cuda.memcpy_dtoh(output_host_mem, output_device_mem)
    predictions.append(float(np.asarray(output_host_mem[0])))

# Lance les inférences
for i in range(nbr_run):
    time0 = time.time()
    np.copyto(input_host_mem,image.ravel())
    cuda.memcpy_htod(input_device_mem, input_host_mem)
    context.execute(batch_size=1,bindings=bindings)
    cuda.memcpy_dtoh(output_host_mem, output_device_mem)
    time_end = time.time()
        
    delais = np.append(delais,time_end - time0)
    predictions.append(float(np.asarray(output_host_mem[0])))
        
    if i%10 == 0:
        print("Etape %d-%d moyenne : %4.1f ms" %(i,i+5,(delais[-10:].mean())*1000))
                       
print("Débit : %.0f images/s" %(nbr_run / delais.sum()))